In [1]:
%%capture
import tensorflow as tf
from tensorflow import keras
from scipy.io import loadmat
from tensorflow.keras import regularizers
import os
import time
import numpy as np
from sklearn.utils import shuffle
import gc
import zipfile
from tensorflow.keras.models import model_from_json
!pip install tqdm
!pip install h5py
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.metrics import r2_score
from sklearn import preprocessing

In [2]:
def scale(x):

    return (x - np.mean(x)) / np.std(x)

In [3]:
def preprocess(x):
    
    return x[:,:,:int(0.75*x.shape[2])]

In [4]:
def cut(x, switch):
    temp = []
    size = int(x.shape[0] / 5)
    for i in range(5):
        temp.append(x[i * size:(i + 1) * size])
    x = np.array(temp)
    
    if switch:
        x = np.reshape(x, (5,-1))
    
    return x

In [5]:
os.chdir("/home/pk2573/Capstone_Data/Re1243Fr20/coarse8")
u = preprocess(scale(loadmat('u_F_xyz_T1.mat')["u_F"]))
v = preprocess(scale(loadmat('v_F_xyz_T1.mat')["v_F"]))
w = preprocess(scale(loadmat('w_F_xyz_T1.mat')["w_F"]))
tau_12 = preprocess(scale(loadmat('tau12_F_xyz_T1.mat')["tau12"]))
tau_13 = preprocess(scale(loadmat('tau13_F_xyz_T1.mat')["tau13"]))
tau_23 = preprocess(scale(loadmat('tau23_F_xyz_T1.mat')["tau23"]))

In [6]:
all_taus = False
significance = True

### Create Datasets

In [7]:
x = np.array([u, v, w])
x = np.transpose(x, [1, 2, 3, 0])
print(x.shape)
x = cut(x, False)
print(x.shape)

(146, 96, 56, 3)
(5, 29, 96, 56, 3)


In [8]:
tau_12 = cut(tau_12, False)
print(tau_12.shape)
tau_12 = np.reshape(tau_12, (tau_12.shape[0], tau_12.shape[1], tau_12.shape[2], tau_12.shape[3], 1))
print(tau_12.shape)
tau_13 = cut(tau_13, False)
tau_13 = np.reshape(tau_13, (tau_13.shape[0], tau_13.shape[1], tau_13.shape[2], tau_13.shape[3], 1))
print(tau_13.shape)
tau_23 = cut(tau_23, False)
tau_23 = np.reshape(tau_23, (tau_23.shape[0], tau_23.shape[1], tau_23.shape[2], tau_23.shape[3], 1))
print(tau_23.shape)

(5, 29, 96, 56)
(5, 29, 96, 56, 1)
(5, 29, 96, 56, 1)
(5, 29, 96, 56, 1)


### Create Model Class 

In [9]:
class CNN(object):
    
    def __init__(self, activation, initializer, regularizer, x_train, y_train, epochs, batch_size, input_shape, filter_size):
        self.activation = activation
        self.initializer = initializer
        self.regularizer = regularizer
        self.x_train = x_train
        self.y_train = y_train
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.filter = filter_size
        
        pass 
    
    def create_model(self):
        model = keras.Sequential([
            tf.keras.layers.Conv3D(128,
                             kernel_size = self.filter,
                             activation = self.activation,
                             input_shape = input_shape,   
                             padding = "same",
                             kernel_regularizer = self.regularizer,
                             data_format = 'channels_last'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv3D(64,
                             kernel_size = (1,1,1),
                             activation = self.activation,
                             padding = "same",
                             kernel_regularizer = self.regularizer),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Conv3D(32,
                             kernel_size = (1,1,1),
                             activation = self.activation,
                             padding = "same",
                             kernel_regularizer = self.regularizer),
             tf.keras.layers.Conv3D(1,
                             kernel_size = (1,1,1),
                             activation = self.activation,
                             padding = "same",
                             kernel_regularizer = self.regularizer),
        ])
        
        return model
    
    def run_model(self):
        model = self.create_model()
        model.compile(optimizer = tf.train.AdamOptimizer(), 
              loss = "mse")
        
        model.summary()
            
        history = model.fit(self.x_train, self.y_train, 
                    epochs = self.epochs, 
                    batch_size =  self.batch_size,
                    verbose = 1)
        
        return history, model

###  Run Model

In [10]:
activation = tf.nn.relu
initializer = None
regularizer = None
epochs = 50
batch_size = 1
filter_size = (7,7,7)

x, tau_12 = shuffle(x, tau_12)
x_train, x_test = x[1:], x[0]
y_train, y_test = tau_12[1:], tau_12[0]

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3], x_train.shape[4])
model = CNN(activation, initializer, regularizer, x_train, y_train, epochs, batch_size, input_shape, filter_size)
history, model = model.run_model()

x_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3]))
y_pred = model.predict(x_test).flatten()

y_true = y_test.flatten()
print("R^2: %.4f" % r2_score(y_true, y_pred))
print("Correlation: %.4f\n" % np.corrcoef(y_pred, y_true)[0, 1])

del x_train
del x_test
del y_train
del y_test
del model
del y_pred
del y_true
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 29, 96, 56, 128)   131840    
_________________________________________________________________
dropout (Dropout)            (None, 29, 96, 56, 128)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 29, 96, 56, 64)    8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 29, 96, 56, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 29, 96, 56, 32)    2080      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 29, 96, 56, 1)     33        
Total params: 142,209
Trainable params: 142,209
Non-trainable params: 0
_________________________________________________________________
Epoc

7